In [3]:
import numpy as np
# from mpl_toolkits import mplot3d
# import matplotlib.pyplot as plt
import random
# plt.style.use('seaborn-poster')

In [4]:
def In(num, a, length):
    for i in range(length):
        if num == a[i]:
            return True
    return False

In [5]:
def Count(infilepath, part_pdg):
    with open(infilepath, 'r') as Input:
        lines = Input.readlines()
    events = 0
    for line in lines:
        if line.startswith('P'):
            pdata = line.split()
#             print(pdata[2])
            if In(pdata[2], part_pdg, len(part_pdg)):
                events += 1
    return events

In [6]:
import numpy as np
import random

def ReadFile(infilepath, outfilepath1, outfilepath3, experiments, charge_dic):
    """
    Reads the input file containing particle data and processes the information.
    Outputs two files: one for training and one for testing based on conditions.

    Parameters:
    - infilepath: Path to the input data file.
    - outfilepath1: Path to the training output file.
    - outfilepath3: Path to the testing output file.
    - experiments: A list to store the processed particle data.
    - charge_dic: A dictionary that maps particle IDs to their charge.
    """
    
    # Define the dtype for particle data for structured numpy arrays
    particle_dtype = np.dtype([
        ('pdgid', np.int32),      # Particle ID
        ('px', np.float64),       # Momentum in x direction
        ('py', np.float64),       # Momentum in y direction
        ('pz', np.float64),       # Momentum in z direction
        ('energy', np.float64),   # Energy of the particle
        ('mass', np.float64),     # Mass of the particle
        ('decay_vertex', np.int32),  # Decay vertex ID
        ('vertex', np.int32)      # Original vertex ID
    ])

    # Open input and output files
    with open(infilepath, 'r') as input_file:
        lines = input_file.readlines()

    # Output files for training and testing data
    with open(outfilepath1, 'w') as train_file, open(outfilepath3, 'w') as test_file:

        cur_ver = []   # Current vertex data
        cur_exp = {}   # Dictionary to hold experiments (vertex -> particles)
        vertex = 0     # Current vertex index
        Bparts = []    # List of B meson decay vertices
        i = 1          # Event counter
        Flag = 0       # Flag to determine whether it's a training or test event

        # Loop through each line of the input file
        for line in lines:
            # Start of a new event
            if line.startswith('E'):
                if vertex != 0:
                    # Store current vertex data in cur_exp dictionary
                    cur_ver_np = np.array(cur_ver, dtype=particle_dtype)
                    cur_exp[vertex] = cur_ver_np
                    experiments.append(cur_exp)  # Add experiment to list
                    cur_exp = {}  # Reset for next event
                    Bparts = []    # Reset B meson parts list
                    vertex = 0     # Reset vertex index

            # New vertex information found
            elif line.startswith('V'):
                if vertex != 0:
                    cur_ver_np = np.array(cur_ver, dtype=particle_dtype)
                    cur_exp[vertex] = cur_ver_np
                cur_ver = []  # Reset for new vertex
                vertex += 1   # Increment vertex counter

            # Particle data line
            elif line.startswith('P'):
                pdata = line.split()

                # Parse particle data
                cur_par = (
                    int(pdata[2]),               # Particle ID (pdgid)
                    np.float64(pdata[3]),         # Momentum in x (px)
                    np.float64(pdata[4]),         # Momentum in y (py)
                    np.float64(pdata[5]),         # Momentum in z (pz)
                    np.float64(pdata[6]),         # Energy
                    np.float64(pdata[7]),         # Mass
                    int(pdata[11]),               # Decay vertex
                    vertex                        # Current vertex ID
                )

                cur_ver.append(cur_par)  # Add particle data to current vertex list

                # Check if the particle is a B+ meson (ID 521)
                if cur_par[0] == 521:
                    # Random decision for Train/Test flag (1 out of 10 chance for Test)
                    ran = random.randint(1, 10)
                    energy = cur_par[4]
                    mass = cur_par[5]
                    px = cur_par[1]
                    py = cur_par[2]
                    pz = cur_par[3]
                    
                    if ran == 1:
                        Flag = 0  # Test
                    else:
                        Flag = 2  # Train

                    # Print particle data into respective output file (Train or Test)
                    if Flag == 2:
                        print('E', px, py, pz, energy, mass, i, file=train_file)
                    else:
                        print('E', px, py, pz, energy, mass, i, file=test_file)

                    Bparts.append(int(pdata[11]))  # Add decay vertex to Bparts
                    i += 1

                # Check if the particle is part of an event (decay within B meson decay)
                if In(-cur_par[7], Bparts, len(Bparts)):
                    Bparts.append(int(pdata[11]))
                    
                    # If particle has no decay (decay_vertex == 0), check its charge
                    if cur_par[6] == 0:
                        ch = charge_dic.get(cur_par[0], 0)  # Get charge from dictionary
                        if Flag == 2:
                            print(ch, end=' ', file=train_file)
                        else:
                            print(ch, end=' ', file=test_file)
                        
                        # Print the particle data
                        for k in cur_par:
                            if Flag == 2:
                                print(k, end=' ', file=train_file)
                            else:
                                print(k, end=' ', file=test_file)

                        if Flag == 2:
                            print('', file=train_file)  # Newline for train file
                        else:
                            print('', file=test_file)  # Newline for test file

            # End of event data (HepMC format)
            if line == 'HepMC::IO_GenEvent-END_EVENT_LISTING':
                cur_ver_np = np.array(cur_ver, dtype=particle_dtype)
                cur_exp[vertex] = cur_ver_np
                experiments.append(cur_exp)
                break  # Stop processing after the last event

In [8]:
# #electron: +11 (charge=-1)
# position: -11 (charge=+1)
# muon: +13 (charge=-1)
# anti-muon: -13 (charge=+1)
# K+: +321 (charge=+1)
# K-: -321 (charge=-1)
# pi+: +211 (charge=+1)
# pi-: -211 (charge=-1)
# proton: 2212 (charge=+1)
# anti-proton: -2212 (charge=-1)
charge_dic = {}
charge_dic[11] = -1
charge_dic[-11] = 1
charge_dic[13] = -1
charge_dic[-13] = 1
charge_dic[321] = 1
charge_dic[-321] = -1
charge_dic[211] = 1
charge_dic[-211] = -1
charge_dic[2212] = 1
charge_dic[-2212] = -1
experiments = []
ReadFile('full_hep_data/hepMCtest', 'raw_data/train1.txt', 'raw_data/test1.txt', experiments, charge_dic)

# Check momentum conservation

In [36]:
def consistent_check(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # Initialize variables before loop
    px_ls = []
    py_ls = []
    pz_ls = []
    energy_ls = []
    mass_ls = []
    charge_ls = []
    pdg_ls = []

    n = 0

    for line in lines:
        if line.startswith('E'):
            if n != 0:
                # Calculate totals
                px_total = sum(px_ls)
                py_total = sum(py_ls)
                pz_total = sum(pz_ls)
                engy_total = sum(energy_ls)

                err = 1e-4
                if abs(px_total - b_meson_px) > err:
                    print("px is not conserved!")
                if abs(py_total - b_meson_py) > err:
                    print("py is not conserved!")
                if abs(pz_total - b_meson_pz) > err:
                    print("pz is not conserved!")
                if abs(engy_total - b_meson_engy) > err:
                    print("energy is not conserved!")
                total_mass = np.sqrt( engy_total**2 - (px_total**2 + py_total**2 + pz_total**2) )
                print(total_mass)
                # Clear lists for next event
                px_ls = []
                py_ls = []
                pz_ls = []
                energy_ls = []
                mass_ls = []
                charge_ls = []
                pdg_ls = []

            # Parse event header line and store B meson data
            exp_inf = line.split()
            b_meson_px = float(exp_inf[1])
            b_meson_py = float(exp_inf[2])
            b_meson_pz = float(exp_inf[3])
            b_meson_engy = float(exp_inf[4])
            b_meson_mass = float(exp_inf[5])

            n = 0

        else:
            par = line.split()
            n += 1
            charge_ls.append(float(par[0]))
            pdg_ls.append(float(par[1]))
            px_ls.append(float(par[2]))
            py_ls.append(float(par[3]))
            pz_ls.append(float(par[4]))
            energy_ls.append(float(par[5]))
            mass_ls.append(float(par[6]))
    return

In [37]:
consistent_check("raw_data/test.txt")

5.279330000000002
5.279329999957599
5.27933
5.279329999999991
5.279330000000002
5.27933
5.279330000000001
5.279330000000002
5.279329999999997
5.279330000000002
5.279330000020478
5.279330000000077
5.279330000000056
5.279330000000002
5.279330000000001
5.279329999999867
5.279329999999998
5.27933
5.27932999999999
5.27933
5.279330000000002
5.279330000000001
5.27933
5.279329999999996
5.279329999999998
5.279330000000005
5.279329999999998
5.27933
5.279329999999999
5.279330000000002
5.279329999992809
5.279330000000096
5.27933
5.279329999999881
5.279329999998112
5.279329999999999
5.279329999999947
5.279329999999998
5.27933
5.279329999999999
5.279330000000001
5.279329999999999
5.279330000000001
5.279330002154609
5.279329999999998
5.279330000001279
5.279329999999998
5.279329999999999
5.279330000000001
5.279329999999999
5.27933
5.2793299999435765
5.279329999999592
5.27933000000733
5.27932999997194
5.279330000000002
5.27933
5.279330000000001
5.279330000000002
5.279329999999998
5.279330000000001
5.27

5.279329999999299
5.279330000000002
5.279330000077745
5.279330000000002
5.2793300000000025
5.279330000181212
5.279329999999998
5.279329999999997
5.279330000147363
5.27933
5.279330000000001
5.279330000000004
5.27933
5.279329999109672
5.279330000000002
5.27933
5.279329999999998
5.279330000000002
5.279330000000002
5.279330000000001
5.279329999999995
5.2793299993648946
5.27933
5.279330000000024
5.27933000017642
5.279330000000215
5.279329999999999
5.279330000000002
5.279330000005585
5.279329999999507
5.279330000000002
5.27933
5.279330000000001
5.279329999999999
5.279329999999999
5.279330000000001
5.279329999999998
5.27933
5.279330000000001
5.279329999999999
5.279330000000001
5.279329999999999
5.279329999999998
5.279329999965479
5.279329999999999
5.279330000000002
5.279329999999999
5.279329998554545
5.2793300000000025
5.27933
5.27933
5.279330000024676
5.279329999999999
5.27933
5.27933
5.279329999999984
5.27933
5.27933
5.279329999998618
5.279329998107904
5.27933
5.279329999999999
5.2793299999

5.279330000000001
5.279329999999998
5.2793300000000425
5.279329999999999
5.279330000003585
5.279330000000009
5.279329999999999
5.279329999999998
5.279330000029976
5.279330000000011
5.27933
5.279329999999997
5.279329999999999
5.279329999999998
5.279329999999996
5.279330000000004
5.279330000000012
5.279329999999999
5.279330000000001
5.279329999999595
5.279330000000002
5.279329999999999
5.279329999999999
5.279329999999999
5.279329999999999
5.27933000000001
5.27933
5.27933000005957
5.27933
5.279330000000001
5.2793299979571335
5.279329999999998
5.279329999999999
5.279329999999999
5.279330000000001
5.279330000000001
5.2793300000000025
5.279330000000002
5.27933
5.279330000000001
5.27933
5.279330000000254
5.27933
5.279330000000001
5.279330000000001
5.27933
5.279329999999998
5.279329999982481
5.279330000912757
5.2793300000000025
5.27933
5.27933
5.279329999999996
5.27933
5.279329999018125
5.279330000000002
5.27933
5.279330000000035
5.279329999999999
5.27933000000006
5.27933
5.27933
5.27933000008